# Drug Response Prediction Using Machine Learning & AI (GDSC)

This notebook executes the complete experiment described in your proposal: predict drug response (IC₅₀) for a **single drug** and **single lineage** using **GDSC** open data, with **Linear Regression** and **Random Forest** models.

**Pipeline**: Download → Load/Explore → Filter (drug/lineage) → Clean/Scale → Train/Evaluate → Plots → Save artifacts.

> Data source: GDSC (CC BY 4.0). See proposal for context and references.


## 1) Setup

Install required libraries (Colab/local).

In [2]:
# If running in Colab, uncomment the line below
# !pip -q install pandas numpy scikit-learn matplotlib requests openpyxl xlrd

import os, io, json, warnings, zipfile, textwrap
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

DATA_DIR = "gdsc_data"
OUT_DIR = "outputs"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

URLS = {
    "expression": "https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Home_files/CellLine_RMA_proc_basalExp.txt",
    "ic50": "https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Home_files/GDSC1_fitted_dose_response_25Feb20.xlsx",
    "cell_lines": "https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Home_files/Cell_Lines_Details.xlsx",
    "compounds": "https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Home_files/Screened_Compounds.xlsx"
}


## 2) Download data (GDSC)

This block downloads the expression matrix (TSV), dose–response (IC₅₀) sheet, cell line metadata, and compound metadata. It is robust to the expression file being plain text (not ZIP).

In [3]:
import requests

def safe_download(url, path):
    r = requests.get(url, timeout=120)
    r.raise_for_status()
    with open(path, "wb") as f:
        f.write(r.content)

print("Downloading GDSC datasets...")
expr_path = os.path.join(DATA_DIR, "CellLine_RMA_proc_basalExp.txt")
if not os.path.exists(expr_path):
    safe_download(URLS["expression"], expr_path)
for key in ["ic50", "cell_lines", "compounds"]:
    p = os.path.join(DATA_DIR, f"{key}.xlsx")
    if not os.path.exists(p):
        safe_download(URLS[key], p)
print("✅ Download complete.")

HTTPError: 404 Client Error: Not Found for url: https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Home_files/CellLine_RMA_proc_basalExp.txt

## 3) Load and quick sanity check

In [ ]:
expr = pd.read_csv(os.path.join(DATA_DIR, "CellLine_RMA_proc_basalExp.txt"), sep="\t")
ic50 = pd.read_excel(os.path.join(DATA_DIR, "ic50.xlsx"))
meta = pd.read_excel(os.path.join(DATA_DIR, "cell_lines.xlsx"))
compounds = pd.read_excel(os.path.join(DATA_DIR, "compounds.xlsx"))

expr.shape, ic50.shape, meta.shape, compounds.shape

## 4) Choose **drug** and **lineage** (from proposal example)

- Drug: *Erlotinib*  
- Lineage: *Lung*

In [ ]:
DRUG_NAME = "Erlotinib"
LINEAGE = "Lung"
DRUG_NAME, LINEAGE

## 5) Filter & merge

- Filter IC₅₀ rows for the selected drug  
- Filter metadata rows for the selected lineage  
- Merge by `COSMIC_ID`  
- Join expression (features)

In [ ]:
# Filter by drug and lineage
ic50_sel = ic50[ic50["DRUG_NAME"].str.contains(DRUG_NAME, case=False, na=False)].copy()
meta_sel = meta[meta["Tissue"].str.contains(LINEAGE, case=False, na=False)].copy()

# Merge dose-response with lineage
df = ic50_sel.merge(meta_sel[["COSMIC_ID","Tissue","Sample Name","Cancer Type"]], on="COSMIC_ID", how="inner")

# Expression has COSMIC_ID as the first column header; rename to match
expr = expr.rename(columns={expr.columns[0]: "COSMIC_ID"})
expr_sel = expr[expr["COSMIC_ID"].isin(df["COSMIC_ID"])]

merged = df.merge(expr_sel, on="COSMIC_ID", how="inner")
merged.shape

### Identify IC₅₀ target column

GDSC files vary in the exact column name. We try common options and fall back to log-transform if needed.

In [ ]:
possible = [c for c in ["LNIC50","LN_IC50","LOG_IC50","IC50"] if c in merged.columns]
if len(possible)==0 and "IC50 (uM)" in merged.columns:
    merged["LN_IC50"] = np.log(merged["IC50 (uM)"].clip(lower=1e-12))
    target_col = "LN_IC50"
elif len(possible)>0:
    target_col = possible[0]
else:
    raise ValueError("No IC50 column detected; inspect 'merged.columns'.")

target_col

### Build feature matrix X and target y

In [ ]:
non_feature = {"COSMIC_ID","Tissue","Sample Name","Cancer Type","DRUG_ID","DRUG_NAME","CELL_LINE_NAME",
               "SCREENING_SET","DATASET","SANGER_MODEL_ID","GDSC1FIT_ID", target_col}
for c in list(merged.columns):
    if "IC50" in c and c != target_col:
        non_feature.add(c)
    if c in ["AUC","RMSE","EINF","HS","SLOPE","R2"]:
        non_feature.add(c)

feature_cols = [c for c in merged.columns if c not in non_feature and merged[c].dtype != "O"]

# Drop feature columns with too many NaNs (>30%)
keep = merged[feature_cols].dropna(axis=1, thresh=int(0.7*len(merged))).columns.tolist()

X = merged[keep].copy()
y = merged[target_col].astype(float).copy()
info = merged[["COSMIC_ID","Sample Name","Tissue","DRUG_NAME"]].copy()

X.shape, y.shape, len(keep)

## 6) Preprocess & split

- Median-impute missing values  
- Standardize features (`StandardScaler`)  
- 80/20 train-test split

In [ ]:
# Median impute
for c in X.columns:
    if X[c].isna().any():
        X[c] = X[c].fillna(X[c].median())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.values)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y.values, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

## 7) Train models

- **Linear Regression**  
- **Random Forest Regressor**

In [ ]:
lin = LinearRegression().fit(X_train, y_train)
rf = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42).fit(X_train, y_train)
lin, rf

## 8) Evaluate & visualize

In [ ]:
def eval_model(model, name):
    y_pred_tr = model.predict(X_train)
    y_pred_te = model.predict(X_test)
    rmse_tr = mean_squared_error(y_train, y_pred_tr, squared=False)
    rmse_te = mean_squared_error(y_test, y_pred_te, squared=False)
    r2_tr = r2_score(y_train, y_pred_tr)
    r2_te = r2_score(y_test, y_pred_te)
    print(f"[{name}] RMSE train={rmse_tr:.3f} | test={rmse_te:.3f}  ||  R2 train={r2_tr:.3f} | test={r2_te:.3f}")
    # Plot
    plt.figure()
    plt.scatter(y_test, y_pred_te, alpha=0.6)
    lims = [min(y_test.min(), y_pred_te.min()), max(y_test.max(), y_pred_te.max())]
    plt.plot(lims, lims, linestyle='--')
    plt.xlabel('Actual (test)'); plt.ylabel('Predicted'); plt.title(f'Predicted vs Actual – {name}')
    os.makedirs(OUT_DIR, exist_ok=True)
    path = os.path.join(OUT_DIR, f'pred_vs_actual_{name.replace(" ","_").lower()}.png')
    plt.savefig(path, bbox_inches='tight'); plt.close()
    return {"model": name, "rmse_train": rmse_tr, "rmse_test": rmse_te, "r2_train": r2_tr, "r2_test": r2_te, "plot": path}

res_lin = eval_model(lin, "Linear Regression")
res_rf  = eval_model(rf, "Random Forest")

res_lin, res_rf

### Random Forest: top-20 feature importances

In [ ]:
importances = rf.feature_importances_
order = np.argsort(importances)[::-1][:20]
top_feats = [X.columns[i] for i in order]
top_vals  = importances[order]

plt.figure(figsize=(8,6))
ypos = np.arange(len(top_feats))[::-1]
plt.barh(ypos, top_vals); plt.yticks(ypos, top_feats)
plt.title('Top 20 Genes by Importance (RF)'); plt.xlabel('Importance')
fi_path = os.path.join(OUT_DIR, 'rf_top_features.png')
plt.tight_layout(); plt.savefig(fi_path, bbox_inches='tight'); plt.close()

fi_path, list(zip(top_feats, top_vals))[:5]

## 9) Save artifacts

In [ ]:
metrics = {"results":[res_lin, res_rf],
           "feature_importance":{"plot": fi_path, "top20_genes": top_feats}}
with open(os.path.join(OUT_DIR, "metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)

final_df = info.copy()
final_df[target_col] = y.values
final_df = pd.concat([final_df, pd.DataFrame(X, columns=X.columns)], axis=1)
csv_path = os.path.join(OUT_DIR, f"filtered_{LINEAGE.lower()}_{DRUG_NAME.lower()}_dataset.csv")
final_df.to_csv(csv_path, index=False)

sorted(os.listdir(OUT_DIR)), csv_path

## 10) Notes & Next steps

- Try different drugs or lineages by changing `DRUG_NAME` and `LINEAGE` above.
- Add cross-validation and hyperparameter tuning (`GridSearchCV`).
- Consider multi-omics features (mutations, CNV, proteomics) to improve signal.
- Report can be compiled from `metrics.json` and saved plots.
